In [1]:
from tqdm.notebook import tqdm
import numpy as np
import sys
sys.path.append('../')
from partitioning import is_pow_of_two, init_membership_list
# from spawn import start_db_background
from structures import Params, NetworkParams
from client_dynamo import client_get, client_put, client_get_memory
from parallel_runner import run_parallel
from random import randint

import logging
logger = logging.getLogger('dynamo_node')
logger.setLevel(logging.INFO)

START_PORT = 2333
CLIENT_ID = 1

def get_start_port(randomize=True):
    return START_PORT + randint(0, params.num_proc-1) * int(randomize)

def get_stats(durations):
    if len(durations) == 0:
        return {}
    durations = np.array(durations) # convert from seconds to ms
    mean = np.mean(durations)
    std = np.std(durations)
    nnth = np.percentile(durations, 99.9)
    return {'mean': mean, 'std': std, '99.9th': nnth}

def generate_plot(durations, label='', clear=True):
    if clear:
        plt.clf()
    fig = sns.distplot(durations, label=label)
    plt.ylabel('Density')
    plt.xlabel('Response Time (in ms)')
    plt.title('Distribution of response times (in ms)')
    plt.legend()
    plt.show()


In [2]:
params = Params({
    'num_proc' : 8,
    'hash_size': 8, # 2^3 = 8 
    'Q' : 16, # 
    'N' : 4,
    'w_timeout': 2,
    'r_timeout': 2,
    'R': 1,
    'W': 3,
    'gossip': False
})

In [3]:
def run_multiple_get(total, num_requests, get_durations):
    for i in tqdm(range(total // num_requests)):
        requests = [client_get]*num_requests
        requests_params = [{'port': get_start_port(), 'client_id': CLIENT_ID, 'key': randint(0, 2**params.hash_size-1)} for _ in range(num_requests)]
        get_durations = np.concatenate((get_durations, run_parallel(requests, requests_params, start_port=START_PORT)))
    return get_durations

def run_multiple_put(total, num_requests, put_durations):
    for i in tqdm(range(total // num_requests)):
        requests = [client_put]*num_requests
        k = randint(0, 2**params.hash_size-1)
        requests_params = [{'port': get_start_port(), 'client_id': CLIENT_ID, 'key': k, 'val': str(k)} for _ in range(num_requests)]
        put_durations = np.concatenate((put_durations, run_parallel(requests, requests_params, start_port=START_PORT)))
    return put_durations

def store_keys(params):
    for key in tqdm(range(2**params.hash_size)):
        client_put(get_start_port(), CLIENT_ID, key=key, val=str(key))

In [35]:
store_keys(params)

## With failures

In [18]:
import logging
logger = logging.getLogger('dynamo_node')
logger.setLevel(logging.INFO)
# logger.propagate = False

In [19]:
def get_start_port(randomize=True, failed_port=None):
    new_port = START_PORT + randint(0, params.num_proc-1) * int(randomize)
    while failed_port is not None and new_port == failed_port:
        new_port = START_PORT + randint(0, params.num_proc-1) * int(randomize)
    return new_port

def run_multiple_get(total, num_requests, get_durations=None, failed_port=None):
    durations, responses = [], []
    for i in tqdm(range(total // num_requests)):
        requests = [client_get]*num_requests
        key = randint(0, 2**params.hash_size-1)
#         while (key >= 176 and key <= 191) or (key >= 208 and key <= 223):
#             # dont want a key to go to a failed coordinator node
#             key = randint(0, 2**params.hash_size-1)                             
        requests_params = [{'port': get_start_port(failed_port=failed_port), 'client_id': CLIENT_ID, 'key': key} for _ in range(num_requests)]
        _durations, _responses = run_parallel(requests, requests_params, start_port=START_PORT, as_np=False)
        print(len(_durations))
        durations.extend(_durations)
        responses.extend(_responses)
    return durations, responses

In [33]:
from client_dynamo import client_fail

START_PORT
mem1, replic_mem = client_get_memory(2335)

keys = []
for k,v in mem1.items():
    keys.append(k)

print(sorted(keys))

keys = []
for _,v in replic_mem.items():
    for k,v in v.mem.items():
        keys.append(k)

print(sorted(keys))

[192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223]
[128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191]


In [22]:
client_fail(START_PORT)

In [ ]:
response = client_put(START_PORT+1, CLIENT_ID, 0, "0")
print(response)

response = client_get(START_PORT+1, CLIENT_ID, 0)
print(response)

In [36]:
# get_fail_durations = np.array([])
get_fail_durations, responses = run_multiple_get(30, 10, None, START_PORT)

10 [0.02781510353088379, 0.013545989990234375, 0.014209747314453125, 0.012731075286865234, 0.016613006591796875, 0.01564192771911621, 0.016098976135253906, 0.01454019546508789, 0.015454292297363281, 0.01527714729309082]
10
10 [0.02606821060180664, 0.014708995819091797, 0.02624201774597168, 0.014147043228149414, 0.01592397689819336, 0.026403188705444336, 0.028207063674926758, 0.025383949279785156, 0.028783082962036133, 0.028288841247558594]
10
10 [0.03347611427307129, 0.05006217956542969, 0.014938831329345703, 0.015185832977294922, 0.03147006034851074, 0.029505014419555664, 0.015604972839355469, 0.029328107833862305, 0.01312398910522461, 0.016121864318847656]
10



In [24]:
get_fail_durations = np.array(get_fail_durations)

get_fail_durations.shape

(30,)

In [25]:
len(get_fail_durations)

30

In [26]:
get_fail_durations*1000

array([15.12098312, 15.65384865, 24.35564995, 29.55174446, 33.38027   ,
       30.64703941, 30.82227707, 31.75878525, 35.39299965, 46.52023315,
       27.48894691, 26.5789032 , 26.01790428, 27.11987495, 30.60102463,
       15.76590538, 31.31079674, 28.49197388, 28.09095383, 13.99278641,
       29.11901474, 29.34479713, 15.58279991, 31.45909309, 27.4810791 ,
       29.53696251, 29.74224091, 28.2497406 , 28.39708328, 29.90221977])

In [28]:
responses

[server_id: 5
 items {
   val: "123"
   context {
     clock {
       server_id: "5"
       count: 1
     }
   }
 }
 metadata: "success"
 reroute_server_id: -1
 succ: true,
 server_id: 5
 items {
   val: "123"
   context {
     clock {
       server_id: "5"
       count: 1
     }
   }
 }
 metadata: "success"
 reroute_server_id: -1
 succ: true,
 server_id: 5
 items {
   val: "123"
   context {
     clock {
       server_id: "5"
       count: 1
     }
   }
 }
 metadata: "success"
 reroute_server_id: -1
 succ: true,
 server_id: 5
 items {
   val: "123"
   context {
     clock {
       server_id: "5"
       count: 1
     }
   }
 }
 metadata: "success"
 reroute_server_id: -1
 succ: true,
 server_id: 5
 items {
   val: "123"
   context {
     clock {
       server_id: "5"
       count: 1
     }
   }
 }
 metadata: "success"
 reroute_server_id: -1
 succ: true,
 server_id: 5
 items {
   val: "123"
   context {
     clock {
       server_id: "5"
       count: 1
     }
   }
 }
 metadata: "success

In [ ]:
get_durations.shape

In [ ]:
test = [0.029925823211669922, 0.03148627281188965, 0.031996965408325195, 0.03176593780517578, 0.031462907791137695, 0.030453920364379883, 0.03252911567687988, 0.0177609920501709, 0.1620798110961914, 0.2121727466583252, 0.22617697715759277, 0.2506279945373535, 0.24697089195251465, 0.29291319847106934, 0.4482302665710449, 0.40848278999328613, 0.48263120651245117]

In [ ]:
len(test)